In [ ]:
import gensim
import pandas as pd
from gensim.models import Word2Vec

In [ ]:
data = pd.read_csv("info.csv")

In [ ]:
asana_dataset = pd.read_csv('Book1.csv')

In [ ]:
data

In [ ]:
asana_dataset['asanas']

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
corpus = []
for i in range(0, 38):
  processed_data = re.sub('[^a-zA-Z]', ' ', data['Benefits'][i])
  processed_data = processed_data.lower()
  processed_data = processed_data.split()
  all_stopwords = stopwords.words('english')
  processed_data = [word for word in processed_data if not word in set(all_stopwords)]
  processed_data = ' '.join(processed_data)
  corpus.append(processed_data)

In [ ]:
for i in range(0,38):
  data['Benefits'][i] = corpus[i]

In [ ]:
data

In [ ]:
final = data.Benefits.apply(gensim.utils.simple_preprocess) #Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.

In [ ]:
final

In [ ]:
len(final)

In [ ]:
connected_dataset = [] #list of asanas Asanas according to the respective number of cleaned words.

In [ ]:
for i in range(len(final)):
  for j in range(len(final[i])):
    connected_dataset.append(asana_dataset['asanas'][i])

In [ ]:
len(connected_dataset)

In [ ]:
connected_dataset

In [ ]:
model = Word2Vec(sentences = final, sg=1, min_count=1, workers=4, window = 5, size = 32)

In [ ]:
model.train(final, total_examples=model.corpus_count, epochs=1000)

(529470, 826000)

In [ ]:
model.wv['peacock']

In [ ]:
len(model.wv.vocab)

In [ ]:
final

In [ ]:
model.wv.word_vec('pain') #another way to get embeddings of a word.

In [ ]:
final[0][0]

In [ ]:
model.wv.most_similar("depression", topn = 10 )

In [ ]:
embeddings_dash = []
for i in range(len(final)):
  for j in final[i]:
    embeddings_dash.append(model.wv[j])

In [ ]:
len(embeddings_dash)

In [ ]:
embeddings_dash[0]

In [ ]:
import numpy as np

In [ ]:
# for i in range(len(embeddings_dash)):
#   embeddings_dash[i] = np.pad(embeddings_dash[i],(6,6)) #when we one hot encoded the asanas, its number is elements were 32. therefore we pad 6 zeros in front and 6 at the back.

In [ ]:
len(embeddings_dash[0])

In [ ]:
embeddings_dash[0]

In [ ]:
final[0][0]

In [ ]:
import numpy as np
embeddings_dash = np.array(embeddings_dash)

In [ ]:
embeddings_dash

In [ ]:
def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot

In [ ]:
arr = one_hot(connected_dataset)

In [ ]:
output = arr

In [ ]:
output

In [ ]:
len(output)

In [ ]:
input = embeddings_dash

In [ ]:
len(input)

In [ ]:
input.shape

In [ ]:
output.shape = input.shape

In [ ]:
output.shape

In [ ]:
import tensorflow as tf

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(220, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=32, activation='softmax'))

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
ann.fit(input, output, batch_size = 5, epochs = 100)

In [ ]:
test = []
test.append(model.wv['pain'])
test.append(model.wv['peacock'])
test = np.array(test)
test

In [ ]:
for i in range(len(asana_dataset['asanas'])):
  for j in ann.predict(test):
    if i == np.argmax(j):
      print(asana_dataset['asanas'][i])